<a href="https://colab.research.google.com/github/anorak-7/ANTISPOOF/blob/main/MODEL_QUANTIZATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Quantization approximates floating-point numbers with lower bit width numbers, dramatically reducing memory footprint and accelerating performance. 

In [ ]:
import os

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

BASED ON PEGASUS, MODEL QUANTIZATION WORKS WELL WITH SEQ2SEQ MODEL AND IS HELPFUL FOR DEPLOYMENT PROCESS

In [ ]:
model_ckpt = "google/pegasus-cnn_dailymail"
model = PegasusForConditionalGeneration.from_pretrained(model_ckpt)
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [ ]:
unquant_model = model.state_dict()
torch.save(unquant_model, "pegasus-unquant.h5")

In [ ]:
model.state_dict().keys()

In [ ]:
quantized_model.config.save_pretrained("pegasus-quantized-config")
quantized_state_dict = quantized_model.state_dict()
torch.save(quantized_state_dict, "pegasus-quantized.h5")

In [ ]:
from transformers import AutoConfig

In [ ]:
config = AutoConfig.from_pretrained("pegasus-quantized-config")
dummy_model = PegasusForConditionalGeneration(config)

In [ ]:
reconstructed_quantized_model = torch.quantization.quantize_dynamic(
    dummy_model, {torch.nn.Linear}, dtype=torch.qint8
)
reconstructed_quantized_model.load_state_dict(quantized_state_dict)

<All keys matched successfully>

In [ ]:
#incase u save the model files locally
quantized_state_dict = torch.load("pegasus-quantized.h5")

In [ ]:
quantized_model.load_state_dict(quantized_state_dict)

<All keys matched successfully>

In [ ]:
quantized_model.state_dict().keys()

In [ ]:
quantized_model

In [ ]:
if model.state_dict().keys() == quantized_model.state_dict().keys():
  print("INTERNAL CONFIGURATION OF LAYERS STAYS CONSTANT")

In [ ]:
unquantized_model_size = os.path.getsize("/content/pegasus-unquant.h5")

In [ ]:
quantized_model_size = os.path.getsize("/content/pegasus-quantized.h5")

In [ ]:
from pathlib import Path

In [ ]:
Path("/content/pegasus-quantized.h5").stat()

os.stat_result(st_mode=33188, st_ino=4589971, st_dev=38, st_nlink=1, st_uid=0, st_gid=0, st_size=973141358, st_atime=1650479353, st_mtime=1650479306, st_ctime=1650479306)

In [ ]:
Path("/content/pegasus-unquant.h5").stat()

os.stat_result(st_mode=33188, st_ino=4589968, st_dev=38, st_nlink=1, st_uid=0, st_gid=0, st_size=2283818289, st_atime=1650479286, st_mtime=1650479295, st_ctime=1650479295)

In [ ]:
print("UNQUANTIZED MODEL SIZE :", unquantized_model_size/(1024*1024),  "MB")
print("QUANTIZED MODEL SIZE   :", quantized_model_size/(1024*1024), "MB")

UNQUANTIZED MODEL SIZE : 2178.0188455581665 MB
QUANTIZED MODEL SIZE   : 928.0599193572998 MB


In [ ]:
if model.config == quantized_model.config:
  print("MODEL ARCHITECTURE IS CONSTANT")


MODEL ARCHITECTURE IS CONSTANT
